In [1]:
import pandas as pd
import numpy as np

### Read data

In [2]:
dataset = pd.read_csv('dataset.csv')
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values
dataset.head()

,ParticipantID,HB,RBC,RET%,RDW-SD,RDW-CV,OFF-HR,LFR,IRF,MFR,HFR
0,1,14.21,5.07,0.89,41.30,13.46,86.21,91.94,8.03,7.16,0.81
1,1,13.96,4.56,0.96,42.46,12.74,81.24,93.08,7.00,6.39,0.51
2,1,14.72,5.13,0.85,41.64,13.16,92.67,89.95,10.04,9.40,0.71
3,1,13.89,4.79,0.92,44.04,13.86,81.53,86.74,13.27,11.79,1.33
4,1,15.42,4.77,1.07,43.28,12.47,92.19,94.42,5.58,4.71,0.75


### Split

In [3]:
# keep profiles separate between train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Create datapoints from train split

In [4]:
N = 20000 # datapoints
prof_max = 20 # maximum no of profiles together in one datapoint
train_len = len(X_train) # total number of individual profiles of patients
feat_len = len(X_train[0]) # number of original features

new_data = np.zeros((N, prof_max*feat_len+1))

for i in range(N):
    # random number of profiles
    n_prof = np.random.randint(2, prof_max)
    # print('No of profiles', n_prof)
    label_list = [] # for assigning label
    for j in range(n_prof):
        # select profile randomly
        idx = np.random.randint(train_len)
        # assign values
        new_data[i, 1+j*feat_len:1+(j+1)*feat_len] = X_train[idx]

        label_list.append(y_train[idx])
    
    # fill values for remaining
    # for j in range(prof_max - n_prof):
    # print('labels', label_list)
    # check if all values are identical in list
    label = label_list.count(label_list[0]) == len(label_list)
    # Assign label
    new_data[i, 0] = label

# concatenate to original    

In [5]:
new_data_df = pd.DataFrame(new_data)
new_data_df    

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,0.0,13.96,4.63,1.81,42.50,13.39,58.49,92.24,7.77,7.09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,14.09,4.55,1.39,39.97,12.13,70.26,95.34,4.74,4.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,14.62,4.66,1.32,44.22,13.18,78.37,92.78,7.12,6.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,13.58,4.58,0.97,44.63,13.90,77.53,87.68,12.45,9.73,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,14.03,4.64,0.98,41.18,12.55,80.75,94.77,5.17,4.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,14.73,4.65,0.88,41.55,12.31,91.68,95.04,4.93,4.39,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,0.0,13.85,4.73,0.72,42.01,13.59,86.97,95.61,4.29,3.65,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,0.0,12.86,4.00,1.84,41.44,12.39,47.16,88.39,11.67,10.41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,0.0,12.98,4.34,1.10,41.53,12.83,66.40,94.28,5.73,5.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_part_cnt = new_data_df.groupby(0).count()
df_part_cnt.iloc[:,:1]

,1
0,
0.0,19949
1.0,51


### Observations
* The new dataset generated is skewed (data imbalance).
* There are very less instances of label=1 (all profile belongs to same class)
* The skewed is obvious because when selecting profiles the probability of selecting other patient profile is more

## Generate additional datapoints with 1 label to balance dataset

In [7]:
# id = np.array(y_train)
# idx = np.where(id == 25)[0]
# # print(len(y_train))
# print(idx)
# # print(y_train)

In [8]:
N_p = int(N*0.9) # datapoints
# N_p = 10
new_data_p = np.zeros((N_p, prof_max*feat_len+1))

for i in range(N_p):

    # random number of profiles
    n_prof = np.random.randint(2, prof_max)

    # random profiles selection
    p = np.random.randint(1, 31)
    
    # indexes of particular participants in data
    p_idx = np.where(np.array(y_train) == p)[0]
    t_len = len(p_idx)
    
    # print('No of profiles', n_prof)
    label_list = [] # for assigning label
    
    for j in range(n_prof):
        # select profile randomly
        idx = np.random.randint(t_len)
    
        # assign values
        new_data_p[i, 1+j*feat_len:1+(j+1)*feat_len] = X_train[p_idx[idx]]

        label_list.append(y_train[p_idx[idx]])
    
    # fill values for remaining
    # for j in range(prof_max - n_prof):
    # print('labels', label_list)
    # check if all values are identical in list
    label = label_list.count(label_list[0]) == len(label_list)
    # Assign label
    new_data_p[i, 0] = label


In [9]:
new_data_df_extra = pd.DataFrame(new_data_p)
new_data_df_extra    

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,1.0,14.17,4.59,1.21,41.59,12.54,76.36,91.76,8.22,7.78,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,13.65,4.49,1.22,40.84,12.48,70.53,95.48,4.56,4.41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,16.26,5.30,1.04,40.80,12.56,101.45,87.39,12.63,11.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,14.68,4.65,0.95,40.89,12.09,89.18,93.75,6.22,5.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,14.01,4.88,1.55,42.27,13.64,66.59,87.72,12.31,10.38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,1.0,14.21,4.85,0.98,41.89,13.07,82.43,92.55,7.45,6.69,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17996,1.0,14.22,4.78,0.75,40.20,12.19,90.75,92.35,7.63,6.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17997,1.0,14.07,4.93,0.89,39.41,12.59,84.48,94.22,5.79,5.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17998,1.0,13.60,4.47,1.00,40.86,12.43,75.46,89.16,10.86,10.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_part_cnt = new_data_df_extra.groupby(0).count()
df_part_cnt.iloc[:,:1]

,1
0,
1.0,18000


In [11]:
new_data.shape, new_data_p.shape

((20000, 201), (18000, 201))

In [12]:
new_data_final = np.vstack((new_data, new_data_p)) # merge original and extra data
new_data_df_final = pd.DataFrame(new_data_final) # create dataframe
new_data_df_final.groupby(0).count().iloc[:,:1] # support for each label


,1
0,
0.0,19949
1.0,18051


In [13]:
# Save as CSV
new_data_df_final.to_csv('train_new.csv', index=None)

In [14]:
# check again
df_train = pd.read_csv('train_new.csv')
df_train

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,0.0,13.96,4.63,1.81,42.50,13.39,58.49,92.24,7.77,7.09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,14.09,4.55,1.39,39.97,12.13,70.26,95.34,4.74,4.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,14.62,4.66,1.32,44.22,13.18,78.37,92.78,7.12,6.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,13.58,4.58,0.97,44.63,13.90,77.53,87.68,12.45,9.73,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,14.03,4.64,0.98,41.18,12.55,80.75,94.77,5.17,4.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37995,1.0,14.21,4.85,0.98,41.89,13.07,82.43,92.55,7.45,6.69,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37996,1.0,14.22,4.78,0.75,40.20,12.19,90.75,92.35,7.63,6.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37997,1.0,14.07,4.93,0.89,39.41,12.59,84.48,94.22,5.79,5.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37998,1.0,13.60,4.47,1.00,40.86,12.43,75.46,89.16,10.86,10.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Create datapoints from testset

In [15]:
N_t = int(N * 0.2) # datapoints
# prof_max = 5 # maximum no of profiles together in one datapoint
test_len = len(X_test) # total number of individual profiles of patients
feat_len = len(X_train[0]) # number of original features

new_data_t = np.zeros((N_t, prof_max*feat_len+1))

for i in range(N_t):
    # random number of profiles
    n_prof = np.random.randint(2, prof_max)
    # print('No of profiles', n_prof)
    label_list = [] # for assigning label
    for j in range(n_prof):
        # select profile randomly
        idx = np.random.randint(test_len)
        # assign values
        new_data_t[i, 1+j*feat_len:1+(j+1)*feat_len] = X_test[idx]

        label_list.append(y_test[idx])
    
    # fill values for remaining
    # for j in range(prof_max - n_prof):
    # print('labels', label_list)
    # check if all values are identical in list
    label = label_list.count(label_list[0]) == len(label_list)
    # Assign label
    new_data_t[i, 0] = label

# concatenate to original    

In [16]:
new_data_df_t = pd.DataFrame(new_data_t)
new_data_df_t    

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,0.0,15.29,5.21,1.01,41.77,13.16,93.00,91.01,8.99,8.58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,12.26,3.75,1.36,43.90,12.35,52.85,81.43,18.56,15.49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,13.85,4.62,1.81,40.35,12.66,57.72,91.72,8.42,7.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,14.42,4.67,1.84,42.79,13.10,62.03,89.58,10.60,9.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,15.75,5.02,0.98,39.77,12.24,98.91,93.10,6.80,5.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.0,12.31,3.77,1.07,42.49,12.28,59.70,91.27,8.66,8.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3996,0.0,13.09,4.29,1.05,43.05,13.10,68.61,90.23,9.75,9.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3997,0.0,13.48,4.43,0.89,38.30,11.82,78.27,96.88,3.19,2.83,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3998,0.0,13.98,4.59,0.94,41.63,12.74,81.46,92.72,7.23,6.58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_part_cnt = new_data_df_t.groupby(0).count()
df_part_cnt.iloc[:,:1]

,1
0,
0.0,3984
1.0,16


### Additional test points to make data balance

In [18]:
N_p = int(N_t*0.9) # datapoints
# N_p = 10
new_data_p = np.zeros((N_p, prof_max*feat_len+1))

for i in range(N_p):

    # random number of profiles
    n_prof = np.random.randint(2, prof_max)

    # random profiles selection
    p = np.random.randint(1, 31)
    
    # indexes of particular participants in data
    p_idx = np.where(np.array(y_test) == p)[0]
    t_len = len(p_idx)
    
    # print('No of profiles', n_prof)
    label_list = [] # for assigning label
    
    for j in range(n_prof):
        # select profile randomly
        idx = np.random.randint(t_len)
    
        # assign values
        new_data_p[i, 1+j*feat_len:1+(j+1)*feat_len] = X_test[p_idx[idx]]

        label_list.append(y_test[p_idx[idx]])
    
    # fill values for remaining
    # for j in range(prof_max - n_prof):
    # print('labels', label_list)
    # check if all values are identical in list
    label = label_list.count(label_list[0]) == len(label_list)
    # Assign label
    new_data_p[i, 0] = label


In [19]:
new_data_p.shape, new_data_p

((3600, 201), array([[ 1.  , 14.97,  5.24, ...,  0.  ,  0.  ,  0.  ],
        [ 1.  , 14.75,  5.01, ...,  0.  ,  0.  ,  0.  ],
        [ 1.  , 13.18,  4.38, ...,  0.  ,  0.  ,  0.  ],
        ...,
        [ 1.  , 14.04,  4.59, ...,  0.  ,  0.  ,  0.  ],
        [ 1.  , 13.89,  4.59, ...,  0.  ,  0.  ,  0.  ],
        [ 1.  , 15.96,  5.44, ...,  0.  ,  0.  ,  0.  ]]))

In [20]:
new_data_t_final = np.vstack((new_data_t, new_data_p)) # merge original and extra data
new_data_df_t_final = pd.DataFrame(new_data_t_final) # create dataframe
new_data_df_t_final.groupby(0).count().iloc[:,:1] # support for each label

,1
0,
0.0,3984
1.0,3616


In [21]:
# Save as CSV
new_data_df_t_final.to_csv('test_new.csv', index=None)